In [5]:
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

DATA_PATH = "/home/xiaohang/Desktop/59970/project/311_Service_Requests_2015.csv" # Make this the /path/to/the/data


In [6]:
col = ['Created Date', 'Incident Zip', 'Complaint Type']
#my_type = {'Created Date': str, 'Incident Zip': float, 'Complaint Type': str}

In [7]:
data_2015 = pd.read_csv(DATA_PATH, usecols=col)

/home/xiaohang/anaconda3/envs/xiaohang/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
data_2015.head()

,Created Date,Complaint Type,Incident Zip
0,03/26/2015 06:50:06 PM,Graffiti,11104
1,03/28/2015 04:30:52 PM,Building/Use,11361
2,04/01/2015 04:48:00 PM,Street Light Condition,NaN
3,05/04/2015 04:46:00 PM,Street Light Condition,11234
4,04/17/2015 01:50:07 PM,Graffiti,11236


In [9]:
data_2015.dtypes

Created Date      object
Complaint Type    object
Incident Zip      object
dtype: object

In [10]:
len(data_2015)

2282098

In [11]:
data_clean = data_2015.dropna(axis=0)

In [12]:
data_clean.describe()

,Created Date,Complaint Type,Incident Zip
count,2114699,2114699,2114699.0
unique,1428636,200,1002.0
top,01/08/2015 12:00:00 AM,HEAT/HOT WATER,11226.0
freq,7244,225042,21284.0


In [13]:
complaint_type = data_clean['Complaint Type']

In [14]:
complaint_type.head(20)

0                          Graffiti
1                      Building/Use
3            Street Light Condition
4                          Graffiti
5            Street Light Condition
7                          Graffiti
8                      Building/Use
9                          Graffiti
10                     Building/Use
11                         Graffiti
12                     Building/Use
13    General Construction/Plumbing
14                         Graffiti
15                     Building/Use
18    General Construction/Plumbing
19                     Building/Use
20                         Graffiti
21          Overgrown Tree/Branches
22                     Building/Use
23          Overgrown Tree/Branches
Name: Complaint Type, dtype: object

In [15]:
# complaint_type = complaint_type.drop_duplicates(subset='Complaint Type', keep='last')
# complaint_type = complaint_type.groupby('Complaint Type')
complaint_type = complaint_type.drop_duplicates()

In [16]:
complaint_type.head(20)

0                                     Graffiti
1                                 Building/Use
3                       Street Light Condition
13               General Construction/Plumbing
21                     Overgrown Tree/Branches
38               Root/Sewer/Sidewalk Condition
69                            New Tree Request
81                              Taxi Complaint
85                       Street Sign - Damaged
96                                Damaged Tree
100                        Special Enforcement
103                   Traffic Signal Condition
113                      Street Sign - Missing
119                               Water System
122                                   Plumbing
129                           Street Condition
131                                   Elevator
140    Special Projects Inspection Team (SPIT)
216                                      Noise
221                                  Dead Tree
Name: Complaint Type, dtype: object

In [17]:
len(complaint_type)

200

In [18]:
data_sample = data_clean[['Created Date', 'Incident Zip']]
data_target = data_clean[['Complaint Type']]

In [19]:
data_sample.head()
data_sample.dtypes

Created Date    object
Incident Zip    object
dtype: object

In [20]:
data_target.head()
data_target.dtypes

Complaint Type    object
dtype: object

In [21]:
complaint_type = complaint_type.reset_index(drop = True)
complaint_type.head()

0                         Graffiti
1                     Building/Use
2           Street Light Condition
3    General Construction/Plumbing
4          Overgrown Tree/Branches
Name: Complaint Type, dtype: object

In [22]:
my_to_str = lambda x:str(x)
data_clean['Complaint Type'] = data_clean['Complaint Type'].astype('str')

/home/xiaohang/anaconda3/envs/xiaohang/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [23]:
data_clean.dtypes

Created Date      object
Complaint Type    object
Incident Zip      object
dtype: object

In [24]:
data_2015['Complaint Type'] = data_2015

In [25]:
complaint_array = complaint_type.values

In [26]:
complaint_array

array(['Graffiti', 'Building/Use', 'Street Light Condition',
       'General Construction/Plumbing', 'Overgrown Tree/Branches',
       'Root/Sewer/Sidewalk Condition', 'New Tree Request',
       'Taxi Complaint', 'Street Sign - Damaged', 'Damaged Tree',
       'Special Enforcement', 'Traffic Signal Condition',
       'Street Sign - Missing', 'Water System', 'Plumbing',
       'Street Condition', 'Elevator',
       'Special Projects Inspection Team (SPIT)', 'Noise', 'Dead Tree',
       'Non-Residential Heat', 'Sewer', 'Air Quality', 'Smoking',
       'Special Natural Area District (SNAD)', 'Illegal Tree Damage',
       'Street Sign - Dangling', 'Sidewalk Condition',
       'For Hire Vehicle Complaint', 'Emergency Response Team (ERT)',
       'Senior Center Complaint', 'Stalled Sites', 'Rodent', 'Electrical',
       'Scaffold Safety', 'Water Conservation', 'Water Quality', 'Lead',
       'Hazardous Materials', 'ELEVATOR', 'Asbestos', 'Plant',
       'Industrial Waste', 'UNSANITARY CONDIT

In [27]:
complaint_dic = {}
count = 0
for i in complaint_array:
    complaint_dic[i] = count
    count+=1
    

In [28]:
complaint_dic

{'AGENCY': 188,
 'APPLIANCE': 100,
 'Adopt-A-Basket': 154,
 'Advocate - Other': 194,
 'Advocate - RPIE': 199,
 'Advocate-Commercial Exemptions': 192,
 'Advocate-Personal Exemptions': 190,
 'Advocate-Prop Class Incorrect': 191,
 'Advocate-SCRIE/DRIE': 189,
 'Air Quality': 22,
 "Alzheimer's Care": 78,
 'Animal Abuse': 124,
 'Animal Facility - No Permit': 152,
 'Animal in a Park': 50,
 'Asbestos': 40,
 'BEST/Site Safety': 44,
 'Beach/Pool/Sauna Complaint': 147,
 'Bereavement Support Group': 166,
 'Bike Rack Condition': 85,
 'Bike/Roller/Skate Chronic': 138,
 'Blocked Driveway': 74,
 'Boilers': 90,
 'Bottled Water': 173,
 'Bridge Condition': 150,
 'Broken Muni Meter': 87,
 'Broken Parking Meter': 134,
 'Building Condition': 183,
 'Building/Use': 1,
 'Bus Stop Shelter Placement': 132,
 'Calorie Labeling': 170,
 'Case Management Agency Complaint': 159,
 'City Vehicle Placard Complaint': 86,
 'Collection Truck Noise': 137,
 'Construction': 73,
 'Consumer Complaint': 84,
 'Cranes and Derricks'

In [29]:
data_clean

,Created Date,Complaint Type,Incident Zip
0,03/26/2015 06:50:06 PM,Graffiti,11104
1,03/28/2015 04:30:52 PM,Building/Use,11361
3,05/04/2015 04:46:00 PM,Street Light Condition,11234
4,04/17/2015 01:50:07 PM,Graffiti,11236
5,05/04/2015 04:41:00 PM,Street Light Condition,11234
7,05/06/2015 12:39:31 AM,Graffiti,11373
8,05/05/2015 10:49:36 AM,Building/Use,10033
9,05/06/2015 01:04:44 PM,Graffiti,10023
10,05/09/2015 10:49:42 AM,Building/Use,11235
11,05/12/2015 07:09:38 AM,Graffiti,11229


In [30]:
import datetime as dt
dates = pd.to_datetime(data_clean['Created Date'], format = ('%I/%d/%Y %H:%M:%S %p'))

In [31]:
dates.apply(lambda x: x.strftime('%I-%d-%Y %H:%M:%S'))

0          03-26-2015 15:50:06
1          03-28-2015 15:30:52
3          05-04-2015 17:46:00
4          04-17-2015 16:50:07
5          05-04-2015 17:41:00
7          05-06-2015 05:39:31
8          05-05-2015 05:49:36
9          05-06-2015 17:04:44
10         05-09-2015 05:49:42
11         05-12-2015 05:09:38
12         05-12-2015 05:54:03
13         05-12-2015 17:42:43
14         05-13-2015 17:52:23
15         05-15-2015 17:51:36
18         05-18-2015 05:49:28
19         05-20-2015 05:18:56
20         06-04-2015 18:22:00
21         05-29-2015 17:22:49
22         06-20-2015 18:18:11
23         06-02-2015 18:30:17
24         06-03-2015 18:20:12
25         06-03-2015 18:12:24
26         06-04-2015 06:14:28
27         06-29-2015 06:50:12
28         06-09-2015 18:49:22
29         07-08-2015 07:23:41
30         06-11-2015 06:25:06
31         07-09-2015 07:39:37
32         07-12-2015 19:40:50
33         06-12-2015 18:12:49
                  ...         
2282066    06-20-2015 06:36:21
2282067 

In [39]:
dates[0].hour

15

In [44]:
len(dates)

2114699

In [45]:
dates.index = range(2114699)

In [46]:
dates.head(30)

0    2015-01-26 15:50:06
1    2015-01-28 15:30:52
2    2015-01-04 17:46:00
3    2015-01-17 16:50:07
4    2015-01-04 17:41:00
5    2015-01-06 05:39:31
6    2015-01-05 05:49:36
7    2015-01-06 17:04:44
8    2015-01-09 05:49:42
9    2015-01-12 05:09:38
10   2015-01-12 05:54:03
11   2015-01-12 17:42:43
12   2015-01-13 17:52:23
13   2015-01-15 17:51:36
14   2015-01-18 05:49:28
15   2015-01-20 05:18:56
16   2015-01-04 18:22:00
17   2015-01-29 17:22:49
18   2015-01-20 18:18:11
19   2015-01-02 18:30:17
20   2015-01-03 18:20:12
21   2015-01-03 18:12:24
22   2015-01-04 06:14:28
23   2015-01-29 06:50:12
24   2015-01-09 18:49:22
25   2015-01-08 07:23:41
26   2015-01-11 06:25:06
27   2015-01-09 07:39:37
28   2015-01-12 19:40:50
29   2015-01-12 18:12:49
Name: Created Date, dtype: datetime64[ns]

In [47]:
date_hours = {}

for i in dates.index:
    date_hours[i] = dates[i].hour

In [48]:
date_hours

{0: 15,
 1: 15,
 2: 17,
 3: 16,
 4: 17,
 5: 5,
 6: 5,
 7: 17,
 8: 5,
 9: 5,
 10: 5,
 11: 17,
 12: 17,
 13: 17,
 14: 5,
 15: 5,
 16: 18,
 17: 17,
 18: 18,
 19: 18,
 20: 18,
 21: 18,
 22: 6,
 23: 6,
 24: 18,
 25: 7,
 26: 6,
 27: 7,
 28: 19,
 29: 18,
 30: 6,
 31: 7,
 32: 19,
 33: 18,
 34: 18,
 35: 19,
 36: 19,
 37: 7,
 38: 19,
 39: 6,
 40: 6,
 41: 8,
 42: 8,
 43: 6,
 44: 6,
 45: 18,
 46: 20,
 47: 21,
 48: 19,
 49: 19,
 50: 21,
 51: 9,
 52: 9,
 53: 21,
 54: 21,
 55: 21,
 56: 21,
 57: 21,
 58: 7,
 59: 19,
 60: 19,
 61: 7,
 62: 19,
 63: 19,
 64: 19,
 65: 7,
 66: 21,
 67: 7,
 68: 21,
 69: 7,
 70: 18,
 71: 21,
 72: 22,
 73: 19,
 74: 10,
 75: 12,
 76: 10,
 77: 20,
 78: 20,
 79: 8,
 80: 20,
 81: 8,
 82: 20,
 83: 22,
 84: 22,
 85: 8,
 86: 8,
 87: 20,
 88: 9,
 89: 9,
 90: 10,
 91: 21,
 92: 22,
 93: 21,
 94: 22,
 95: 21,
 96: 9,
 97: 10,
 98: 22,
 99: 22,
 100: 22,
 101: 22,
 102: 22,
 103: 22,
 104: 22,
 105: 22,
 106: 22,
 107: 23,
 108: 11,
 109: 10,
 110: 22,
 111: 22,
 112: 22,
 113: 23,
 114:

In [34]:
temp = data_clean.values

In [35]:
temp

array([['03/26/2015 06:50:06 PM', 'Graffiti', 11104.0],
       ['03/28/2015 04:30:52 PM', 'Building/Use', 11361.0],
       ['05/04/2015 04:46:00 PM', 'Street Light Condition', 11234.0],
       ..., 
       ['09/02/2015 12:58:00 PM', 'Sewer', 11001.0],
       ['09/02/2015 08:59:00 AM', 'Sewer', 11691.0],
       ['07/16/2015 09:42:04 AM', 'General Construction/Plumbing', 11222.0]], dtype=object)

In [73]:
temp[0][:]

array([30264625, '03/26/2015 06:50:06 PM', 11104.0, 'DSNY',
       'Department of Sanitation', 'Graffiti'], dtype=object)

In [74]:
temp2 = data_clean.to_records()

In [75]:
temp2

rec.array([ (0, 30264625, '03/26/2015 06:50:06 PM', 11104.0, 'DSNY', 'Department of Sanitation', 'Graffiti'),
 (1, 30279915, '03/28/2015 04:30:52 PM', 11361.0, 'DOB', 'Department of Buildings', 'Building/Use'),
 (2, 30314570, '04/01/2015 04:48:00 PM', nan, 'DOT', 'Department of Transportation', 'Street Light Condition'),
 ...,
 (2282095, 33435251, '09/02/2015 12:58:00 PM', 11001.0, 'DEP', 'Department of Environmental Protection', 'Sewer'),
 (2282096, 33458138, '09/02/2015 08:59:00 AM', 11691.0, 'DEP', 'Department of Environmental Protection', 'Sewer'),
 (2282097, 33983441, '07/16/2015 09:42:04 AM', 11222.0, 'DOB', 'Department of Buildings', 'General Construction/Plumbing')], 
          dtype=[('index', '<i8'), ('Unique Key', '<i8'), ('Created Date', 'O'), ('Incident Zip', '<f8'), ('Agency', 'O'), ('Agency Name', 'O'), ('Complaint Type', 'O')])